# Programación Dinámica
- Francisco Castillo - 21562
- Diego Lemus - 21469

## Task 1


### ¿Qué es la Programación Dinámica y cómo se relaciona con Reinforcement Learning?
Es un paradigma de programación que se utiliza para resolver problemas que pueden descomponerse en otros subproblemas más simples. En los Markov Decision Processes, la programación dinámica permite encontrar la política óptima y el valor de los estados mediante la iteración de políticas o la iteración de valores mientras se conozcan las dinámicas del entorno (probabilidades de transición y recompensas).
### Explique en sus propias palabras el algoritmo de Iteración de Póliza
El algoritmo de Policy Iteration es un método para encontrar la política óptima en un MDP. Consiste en dos pasos:
1. **Evaluación de la política:** se calcula el valor de los estados bajo la política actual.
2. **Mejora de la política:** se actualiza la política seleccionando acciones que maximicen el valor esperado de los estados. Este proceso se repite hasta que la política converge.
### Explique en sus propias palabras el algoritmo de Iteración de Valor
El algoritmo de Value Iteration es un método para encontrar la política óptima en un MDP mediante la actualización iterativa de los valores de los estados. Este solo tiene un paso:
1. **Actualización de valores:** se actualizan los valores de los estados utilizando la ecuación de Bellman, considerando las acciones posibles y las probabilidades de transición. Este proceso se repite hasta que los valores convergen, lo que permite derivar la política óptima a partir de los valores finales.
### En el laboratorio pasado, vimos que el valor de los premios obtenidos se mantienen constantes, ¿por qué?
Se mantienen constantes porque era un ambiente determinista con recomepnsas fijas. Esto implica que la función de recomensa no cambia con el tiempo y las recompensas son predecibles.

> [Value Iteration vs. Policy Iteration](https://www.geeksforgeeks.org/data-science/what-is-the-difference-between-value-iteration-and-policy-iteration/)

## Task 2

In [1]:
from obj.policy import Policy
from obj.maze import Maze
from obj.simulator import Simulator
from obj.mdp_solver import MDP_Solver

In [2]:
maze = Maze()
policy = Policy()
simulator = Simulator(maze, policy)

In [3]:
print("\n=== Política dada ===")
policy.print_policy()
simulator.simulate_policy_trace()
avg = simulator.evaluate_policy(runs=100)
print(f"Recompensa promedio tras 100 simulaciones (política dada): {avg:.2f}\n")


=== Política dada ===
Política π (estado → acción):

  Estado 0: 'DOWN'

  Estado 1: 'LEFT'

  Estado 2: 'LEFT'

  Estado 3: 'DOWN'

  Estado 4: 'LEFT'

  Estado 5: 'DOWN'

  Estado 6: 'RIGHT'

  Estado 7: 'RIGHT'

  Estado 8: 'UP'

Simulación paso a paso:
 Paso 1: 0 --[DOWN]--> 3 | Recompensa = 0
 Paso 2: 3 --[DOWN]--> 6 | Recompensa = 0
 Paso 3: 6 --[RIGHT]--> 7 | Recompensa = 0
 Paso 4: 7 --[RIGHT]--> 8 | Recompensa = 100
Recompensa acumulada total: 100

Recompensa promedio tras 100 simulaciones (política dada): 100.00



### Iteración de Valor

In [4]:
solver_vi = MDP_Solver(maze, gamma=0.9, theta=0.001)
V_opt = solver_vi.value_iteration()
pi_from_vi = solver_vi.extract_policy()

print("=== Resultados Iteración de Valor ===")
print("Función de valor óptima V*:")
for s in sorted(V_opt):
    print(f"  V({s}) = {V_opt[s]:.3f}")
print("\nPolítica óptima extraída π* (de V*):")
for s in sorted(pi_from_vi):
    action = pi_from_vi[s] or "-"
    print(f"  Estado {s}: {action}")

=== Resultados Iteración de Valor ===
Función de valor óptima V*:
  V(0) = 72.900
  V(1) = 65.610
  V(2) = 90.000
  V(3) = 81.000
  V(4) = 90.000
  V(5) = 100.000
  V(6) = 90.000
  V(7) = 100.000
  V(8) = 0.000

Política óptima extraída π* (de V*):
  Estado 0: DOWN
  Estado 1: LEFT
  Estado 2: DOWN
  Estado 3: DOWN
  Estado 4: DOWN
  Estado 5: DOWN
  Estado 6: RIGHT
  Estado 7: RIGHT
  Estado 8: -


#### Análisis

- Estados cercanos a la meta (5,7) alcanzan el valor máximo descontado de 100.
- El estado terminal 8 tiene V*(8)=0 porque allí termina el proceso.
- Un agente que empiece en el estado 0 puede esperar en promedio unas 72.9 unidades de recompensa descontadas antes de llegar a la meta si sigue la política óptima.

### Iteración de Política

In [5]:
solver_pi = MDP_Solver(maze, gamma=0.9, theta=0.001)
pi_opt = solver_pi.policy_iteration()
V_from_pi = solver_pi.V

print("=== Resultados Iteración de Política ===")
print("Política óptima π* (policy iteration):")
for s in sorted(pi_opt):
    action = pi_opt[s] or "-"
    print(f"  Estado {s}: {action}")
print("\nFunción de valor resultante V(s) para π*:")
for s in sorted(V_from_pi):
    print(f"  V({s}) = {V_from_pi[s]:.3f}")

=== Resultados Iteración de Política ===
Política óptima π* (policy iteration):
  Estado 0: DOWN
  Estado 1: LEFT
  Estado 2: DOWN
  Estado 3: DOWN
  Estado 4: DOWN
  Estado 5: DOWN
  Estado 6: RIGHT
  Estado 7: RIGHT
  Estado 8: RIGHT

Función de valor resultante V(s) para π*:
  V(0) = 72.900
  V(1) = 65.610
  V(2) = 90.000
  V(3) = 81.000
  V(4) = 90.000
  V(5) = 100.000
  V(6) = 90.000
  V(7) = 100.000
  V(8) = 0.000


#### Análisis

- Policy Iteration en general converge en menos iteraciones que Value Iteration.
- La política final coincide prácticamente con la extraída por Value Iteration.
- Se confirma que π* es la misma política óptima hallada por ambos métodos para el MDP.

### Conclusiones

Modelado del MDP: las matrices PP y RR están bien definidas, por ende las transiciones deterministas y recompensas son correctas.


Política óptima única: los dos métodos convergen al mismo resultado, lo que demuestra que la política encontrada es, de hecho, la óptima para este problema.


Eficacia de la simulación: al usar π* en el simulador, se obtendrá siempre la máxima recompensa esperada, sin riesgo de choque.